In [4]:
import numpy as np
from urllib.parse import quote
from flatten_json import flatten
import urllib.request
from bs4 import BeautifulSoup
import requests
import time, re, json, copy, os, random
import shutil
from pprint import pprint
from collections import defaultdict, Counter

In [12]:
USER_AGENT_LIST = [
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
            "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
            "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
            "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
            "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
            "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
            "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
            "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 LBBROWSER",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; 360SE)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8C148 Safari/6533.18.5",
            "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b13pre) Gecko/20110307 Firefox/4.0b13pre",
            "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
            "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
        ]

In [2]:
with open("/home/yingshac/CYS/WebQnA/WebQnA_data/img_metadata-Copy1.json", "r") as f:
    img_meta = json.load(f)
print(len(img_meta))

362367


In [43]:
import unicodedata
import string
TABLE = str.maketrans(dict.fromkeys(string.punctuation)) 
def simplify(text):
    
    try:
        text = unicode(text, 'utf-8')
    except NameError:
        pass
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8").strip()
    return str(text)

In [113]:
def get_description_by_url(desurl):
        req = urllib.request.Request(desurl, headers = {'User-Agent': random.choice(USER_AGENT_LIST)})
        try:
            with urllib.request.urlopen(req) as f:
                html = f.read().decode('utf-8')
        except:
            return "NOTFOUND"

        soup = BeautifulSoup(html, 'html.parser')
        des = ""
        result = soup.find_all('td',class_= "description")
        
        for r in result:
            row = r.text.strip()
            if row[:8] == 'English:':
                row = row[8:]
            des += simplify(row) + " "
        des = des.strip()
        
        if len(des) == 0:
            result = soup.find_all('div', class_="description mw-content-ltr en")
            for r in result:
                row = r.text.strip()
                if row[:8] == 'English:':
                    row = row[8:]
                des += simplify(row) + " "
            des = des.strip()
        return des

In [76]:
result = {}
count = 0
before = 0
after = 0
for k in sorted(list(img_meta.keys()))[290000:290200]:
    result[k] = copy.deepcopy(img_meta[k])
    count += 1
    if len(img_meta[k]['description'].strip()) == 0:
        before += 1
        x = get_description_by_url(img_meta[k]['page'])
        if x == "NOTFOUND" or len(x.translate(TABLE).strip())==0: 
            x = ""
        result[k]['description'] = x
        if len(x) == 0: after += 1

    if count % 100 == 0:
        print(count, before, after)
        before = 0
        after = 0
        #json.dump(result, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_meta_upd/img_meta_{}.json".format(args.boundary), "w"), indent=4)

100 91 3
200 88 11


In [115]:
get_description_by_url("https://commons.wikimedia.org/wiki/File:Rock_Choir_at_Easton_Lodge_Gardens_open_day%2C_Little_Easton%2C_Essex%2C_England_01.jpg")

'The Rock Choir on the croquet lawn at Easton Lodge Gardens in Little Easton, Essex, England, on 20 May 2018.Camera: Canon EOS 6D Mark II with Canon EF 24-105mm F4L IS USM lens.Software: File lens-corrected, optimized, perhaps cropped, with DxO PhotoLab, and likely further optimized with Adobe Photoshop CS2.'

In [144]:
### check progress
path = "/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_meta_upd/"
#print(os.listdir(path))
img_meta_upd = {}
if os.path.isdir(path):
    finished_samples = []
    files = os.listdir(path)
    print("{} files found".format(len(files)))
    for f in files:
        if not '.json' in f: continue
        try: img_meta_upd.update(json.load(open(os.path.join(path, f), "r")))
        except: raise
else:
    img_meta_upd.update(json.load(open(path, "r")))
print("{} samples found".format(len(img_meta_upd)))
description_lengths = [len(img_meta_upd[k]['description']) for k in img_meta_upd if len(img_meta_upd[k]['description'])<10]
print(len(description_lengths), Counter(description_lengths))

13 files found
362341 samples found
33295 Counter({0: 28725, 8: 1267, 9: 802, 6: 738, 7: 561, 5: 507, 3: 325, 4: 290, 1: 43, 2: 37})


In [123]:
random.sample(list(img_meta_upd.keys()), 30)

['10117614',
 '10277709',
 '10320351',
 '10024335',
 '7041',
 '10067871',
 '2096',
 '10263924',
 '10014408',
 '10169947',
 '10016815',
 '10089871',
 '10204065',
 '10279648',
 '10079424',
 '10187984',
 '21637',
 '10166634',
 '10208720',
 '10024987',
 '10150909',
 '10025510',
 '10044019',
 '10333051',
 '10129747',
 '10053200',
 '10041936',
 '10017529',
 '10078175',
 '10153839']

In [112]:
for k in img_meta_upd:
    if len(img_meta_upd[k]['description']) == 0 and random.random()<0.0001:
        print(img_meta_upd[k]['page'])

https://commons.wikimedia.org/wiki/File:A_strange_but_familiar_world_(5730783407).jpg
https://commons.wikimedia.org/wiki/File:1024_Botanischer_Garten_%E2%80%9AAlpinum_Juliana%E2%80%98-2691.jpg
https://commons.wikimedia.org/wiki/File:Central_Park_NYC_-_'Indian_Hunter'_Statue_by_John_Quincy_Adams_Ward_-_IMG_5711.JPG
https://commons.wikimedia.org/wiki/File:Roadside_Station_%2CSAGAE_2013.jpg
https://commons.wikimedia.org/wiki/File:State_Department_Images_WTC_9-11_Prayer_Meeting.jpg
https://commons.wikimedia.org/wiki/File:Festuca_pratensis_-_Botanischer_Garten_M%C3%BCnchen-Nymphenburg_-_DSC07799.JPG
https://commons.wikimedia.org/wiki/File:%C3%89glise_Saint-Pierre-%C3%A8s-Liens_de_Collongues_2.jpg
https://commons.wikimedia.org/wiki/File:Lotto%2C_nozze_mistiche_bergamo_00.jpg


### Check description lengths for x_distractors

In [96]:
txt_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0823_clean_te.json", "r"))
print(Counter([txt_dataset[k]['split'] for k in txt_dataset]))
print(len(set([txt_dataset[k]['Guid'] for k in txt_dataset])))

Counter({'train': 17812, 'test': 4076, 'val': 2455})
24343


In [99]:
for k in txt_dataset:
    for im in txt_dataset[k]['img_negFacts']:
        if len(im['caption'])<5:
            print(im['caption']) 

Ra. 
Ra. 
War
Red
Hu. 
40. 
40. 
War
2013
M198
 Ion
War
 Ch
Ægir
Area
Kick
Ali
Red
 Ion
Area
Si–X
Area
 Ion
 Ion
 Ion
Area
Type
F3. 
F3. 
Si–X
Type
War
War
Ali
 Ion
Type
Si–X
 Ion
War
2
Kick
